# Neural Graph Collaborative Filtering (NGCF)

Neural Graph Collaborative Filtering (NGCF) is a recommendation algorithm that uses a graph-based approach to improve recommendation quality. It constructs a user-item bipartite graph and refines user and item embeddings through a graph neural network. This process captures complex user-item relationships by considering high-order connectivity in the graph, leading to more accurate and relevant recommendations. NGCF is particularly effective in scenarios with rich and intricate user-item interaction data.

This is a TensorFlow implementation of NGCF with a custom training loop.

Neural Graph Collaborative Filtering (NGCF) is a state-of-the-art GCN-based recommender model that takes advantage of graph structure and is a precursor to the superior LightGCN. In this notebook, we construct and train an NGCF model and evaluate its performance.

In [1]:
import sys
import os
# Append the parent directory to sys.path for relative imports
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

import math
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Progbar
from src.utils import preprocess, metrics
from src.models import NGCF

# Suppress warnings for cleaner notebook presentation
import warnings
warnings.simplefilter("ignore")

## Prepare data

This NGCF implementation takes an adjacency matrix in a sparse tensor format as input.

In preparation of the data for NGCF, we must:

* Stratified train test split
* Create a normalized adjacency matrix
* Convert to tensor
* Load data

In [2]:
# Loading ratings data
rating_file = os.path.join('..', 'src', 'data', 'goodreads_2m', 'ratings.csv')
ratings = pd.read_csv(rating_file)

# Displaying the shape of the dataset and a random sample of 5 entries
print(f'Shape: {ratings.shape}')
ratings.sample(5, random_state=123)  # Setting a seed for reproducibility

Shape: (91226, 3)


,user_id,book_name,rating
74505,2540,"A Game of Thrones (A Song of Ice and Fire, #1)",4
60643,5886,The Amazing Adventures of Kavalier & Clay,4
87603,4411,The World to Come,5
81524,4934,Harry Potter and the Philosopher's Stone (Harr...,5
60556,5791,"Bloodsucking Fiends (A Love Story, #1)",3


## Train test split

We split the data using a stratified split so the users in the training set are also the same users in the test set. NGCF is not able to generate recommendations for users not yet seen in the training set.

In [3]:
# Splitting the data into train and test sets
train_size = 0.75
train, test = preprocess.stratified_split(ratings, 'user_id', train_size)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

# Displaying train and test set details
print(f'Train Set Shape: {train.shape}')
print(f'Test Set Shape: {test.shape}')
print(f'Do they have the same users?: {set(train.user_id) == set(test.user_id)}')

# Combining train and test data for global statistics
combined = pd.concat([train, test]).reset_index(drop=True)
n_users = combined['user_id'].nunique()
n_books = combined['book_name'].nunique()

# Displaying global statistics
print(f'Number of Users: {n_users}')
print(f'Number of Books: {n_books}')


Train Set Shape: (68435, 3)
Test Set Shape: (22791, 3)
Do they have the same users?: True
Number of Users: 1371
Number of Books: 2720


## Reindex

Reset the index of users and books from 0-n for both the training and test data. This is to allow better tracking of users and books. Dictionaries are created so we can easily translate back and forth from the old index to the new index.

In [4]:
# Reindexing users and books
unique_user_ids = combined['user_id'].unique()
user2id = {user_id: i for i, user_id in enumerate(unique_user_ids)}

# Create a new DataFrame for unique books with new indices
book_new = combined[['book_name']].drop_duplicates().reset_index(drop=True)
book_new['book_name_new'] = np.arange(len(book_new))

# Function to reindex DataFrame
def reindex_df(df, book_new, user2id):
    df_reindex = pd.merge(df, book_new, on='book_name', how='left')
    df_reindex['user_id_new'] = df['user_id'].map(user2id)
    return df_reindex[['user_id_new', 'book_name_new', 'rating']]

# Applying the reindexing to train and test sets
train_reindex = reindex_df(train, book_new, user2id)
test_reindex = reindex_df(test, book_new, user2id)

# Creating mapping dictionaries
item2id = dict(zip(book_new['book_name'], book_new['book_name_new']))
id2item = dict(zip(book_new['book_name_new'], book_new['book_name']))
id2user = dict(zip(train_reindex['user_id_new'], train['user_id']))

# Grouping interacted items by users
interacted = train_reindex.groupby("user_id_new")["book_name_new"].apply(set).reset_index().rename(columns={"book_name_new": "book_interacted"})

## Adjacency matrix

In our case, nodes are both users and books. Rows and columns consist of ALL the nodes and for every connection (reviewed book) there is the value 1.

To first create the adjacency matrix we first create a user-item graph where similar to the adjacency matrix, connected users and books are represented as 1 in a sparse array. Unlike the adjacency matrix, a user-item graph only has users for the columns/rows and items as the other, whereas the adjacency matrix has both users and items concatenated as rows and columns.

In this case, because the graph is undirected (meaning the connections between nodes do not have a specified direction) the adjacency matrix is symmetric. We use this to our advantage by transposing the user-item graph to create the adjacency matrix.

Our adjacency matrix will not include self-connections where each node is connected to itself.

## Create adjacency matrix

In [5]:
# Creating user-item graph as a sparse matrix
R = sp.dok_matrix((n_users, n_books), dtype=np.float32)
for _, row in train_reindex.iterrows():
    R[row['user_id_new'], row['book_name_new']] = 1

# Creating the adjacency matrix
adj_mat = sp.dok_matrix((n_users + n_books, n_users + n_books), dtype=np.float32)
adj_mat[:n_users, n_users:] = R
adj_mat[n_users:, :n_users] = R.T

## Normalize adjacency matrix

This helps numerically stabilize values when repeating graph convolution operations, avoiding the scale of the embeddings increasing or decreasing.

In [6]:
# Creating the degree matrix
D_values = np.array(adj_mat.sum(1))
D_inv_values = np.power(D_values + 1e-9, -0.5).flatten()
D_inv_values[np.isinf(D_inv_values)] = 0.0
D_inv_sq_root = sp.diags(D_inv_values)

# Normalizing the adjacency matrix
norm_adj_mat = D_inv_sq_root.dot(adj_mat).dot(D_inv_sq_root)

### Convert to Tensor

In [7]:
# Convert the normalized adjacency matrix to COO format for TensorFlow SparseTensor
coo = norm_adj_mat.tocoo().astype(np.float32)
indices = np.hstack((coo.row[:, np.newaxis], coo.col[:, np.newaxis]))
A_tilde = tf.SparseTensor(indices, coo.data, coo.shape)

## NGCF

### Custom training

In [8]:
# Model configuration
N_LAYERS = 5
EMBED_DIM = 64
DECAY = 0.0001
EPOCHS = 50
BATCH_SIZE = 1024
LEARNING_RATE = 1e-2

# Expected number of parameters
print(f'Parameters: {EMBED_DIM**2 + EMBED_DIM * (n_users + n_books)}')

# Initialize the NGCF model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model = NGCF.NGCF(adj_mat=A_tilde, R=R, n_users=n_users, n_items=n_books, n_layers=N_LAYERS, emb_dim=EMBED_DIM, decay=DECAY)

Parameters: 265920


In [9]:
%%time

def sample_neg(interacted_items, n_books):
    """Function to sample a negative item not interacted by the user"""
    while True:
        neg_item = random.randint(0, n_books - 1)
        if neg_item not in interacted_items:
            return neg_item

# Training loop
for epoch in range(1, EPOCHS + 1):
    print(f'Epoch {epoch}/{EPOCHS}')
    n_batch = train_reindex.shape[0] // BATCH_SIZE + (train_reindex.shape[0] % BATCH_SIZE != 0)
    bar = Progbar(n_batch)

    for _ in range(n_batch):
        # Sample a batch of users
        users = np.random.choice(n_users, BATCH_SIZE, replace=False)

        # Sample positive and negative items for each user
        pos_items = [random.choice(list(interacted.loc[user]['book_interacted'])) for user in users]
        neg_items = [sample_neg(interacted.loc[user]['book_interacted'], n_books) for user in users]

        with tf.GradientTape() as tape:
            # Get new embeddings
            new_user_embeddings, new_item_embeddings = model(model.user_embedding, model.item_embedding)
            # Look up embeddings for sampled users and items
            user_embeddings = tf.nn.embedding_lookup(new_user_embeddings, users)
            pos_item_embeddings = tf.nn.embedding_lookup(new_item_embeddings, pos_items)
            neg_item_embeddings = tf.nn.embedding_lookup(new_item_embeddings, neg_items)

            # Look up old embeddings for regularisation term
            old_user_embeddings = tf.nn.embedding_lookup(model.user_embedding, users)
            old_pos_item_embeddings = tf.nn.embedding_lookup(model.item_embedding, pos_items)
            old_neg_item_embeddings = tf.nn.embedding_lookup(model.item_embedding, neg_items)

            # Compute scores and loss
            pos_scores = tf.reduce_sum(user_embeddings * pos_item_embeddings, axis=1)
            neg_scores = tf.reduce_sum(user_embeddings * neg_item_embeddings, axis=1)
            mf_loss = tf.reduce_mean(tf.nn.softplus(neg_scores - pos_scores))
            emb_loss = DECAY * (tf.nn.l2_loss(old_user_embeddings) + tf.nn.l2_loss(old_pos_item_embeddings) + tf.nn.l2_loss(old_neg_item_embeddings)) / BATCH_SIZE
            loss = mf_loss + emb_loss

            # Compute gradients and apply them
            grads = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

            bar.add(1, values=[('training loss', float(loss))])


Epoch 1/50
67/67 [==============================] - 7s 109ms/step - training loss: 0.5011
Epoch 2/50
67/67 [==============================] - 7s 107ms/step - training loss: 0.3943
Epoch 3/50
67/67 [==============================] - 7s 107ms/step - training loss: 0.3137
Epoch 4/50
67/67 [==============================] - 7s 109ms/step - training loss: 0.2799
Epoch 5/50
67/67 [==============================] - 8s 116ms/step - training loss: 0.2563
Epoch 6/50
67/67 [==============================] - 7s 108ms/step - training loss: 0.2273
Epoch 7/50
67/67 [==============================] - 8s 115ms/step - training loss: 0.2136
Epoch 8/50
67/67 [==============================] - 7s 112ms/step - training loss: 0.2013
Epoch 9/50
67/67 [==============================] - 7s 108ms/step - training loss: 0.1896
Epoch 10/50
67/67 [==============================] - 7s 106ms/step - training loss: 0.1853
Epoch 11/50
67/67 [==============================] - 8s 116ms/step - training loss: 0.1828
Epoch 12

## Recommend

In [10]:
# Make recommendations
# Convert test user ids to the new ids
users = np.array([user2id[x] for x in test['user_id'].unique()])
recommendations = model.recommend(users, k=10)
recommendations = recommendations.replace({'user_id': id2user, 'book_name': id2item})
recommendations.head(5)

,user_id,book_name,prediction
0,1,Into the Wild,10.798458
1,1,Predictably Irrational: The Hidden Forces That...,10.849976
2,1,The Tipping Point: How Little Things Can Make ...,14.096086
3,1,"The Da Vinci Code (Robert Langdon, #2)",14.454141
4,1,Brave New World,12.045841


## Evaluation Metrics

The performance of our model is evaluated using the test set, which consists of the exact same users in the training set but with books the users have reviewed that the model has not seen before. A good model will recommend books that the user has also reviewed in the test set.

---

### Precision@k

Out of the books that are recommended, what proportion is relevant. Relevant in this case is if the user has reviewed the book.

A precision@10 of about 0.1 means that about 10% of the recommendations are relevant to the user. In other words, out of the 10 recommendations made, on average a user will have 1 book that is actually relevant.

### Recall@k

Out of all the relevant books (in the test set), how many are recommended.

A recall@10 of 0.1 means that about 10% of the relevant books were recommended. By definition you can see how even if all the recommendations made were relevant, recall@k is capped by k. A higher k means that more relevant books can be recommended.

### Mean Average Precision (MAP)

Calculate the average precision for each user and average all the average precisions over all users. Penalizes incorrect rankings of books.

### Normalized Discounted Cumulative Gain (NDGC)

Looks at both relevant books and the ranking order of the relevant books. Normalized by the total number of users.

---


In [11]:
# Evaluate model performance
top_k = recommendations.copy()
top_k['rank'] = top_k.groupby('user_id', sort=False).cumcount() + 1

# Calculate evaluation metrics
precision_at_k = metrics.precision_at_k(top_k, test, 'user_id', 'book_name', 'rank')
recall_at_k = metrics.recall_at_k(top_k, test, 'user_id', 'book_name', 'rank')
mean_average_precision = metrics.mean_average_precision(top_k, test, 'user_id', 'book_name', 'rank')
ndcg = metrics.ndcg(top_k, test, 'user_id', 'book_name', 'rank')

# Display evaluation metrics
print(f'Precision: {precision_at_k:.6f}',
      f'Recall: {recall_at_k:.6f}',
      f'MAP: {mean_average_precision:.6f}',
      f'NDCG: {ndcg:.6f}', sep='\n')

Precision: 0.153611
Recall: 0.111269
MAP: 0.038892
NDCG: 0.151472
